In [1]:
from pprint import pprint
from time import sleep
import requests
from bs4 import BeautifulSoup
from .. _partecipanti.gestione_input import leggi_float

# url_list = [
#     'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=1',
#     'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=2'
# ]

base_url = 'https://www.subito.it/annunci-italia/vendita/usato/'


def scrape_page(page):
    query_str = {
        'q': 'spectrum+sinclair',
        'o': page
    }

    result = requests.get(base_url, params=query_str)

    if result.status_code == 200:
        soup = BeautifulSoup(result.content, 'html.parser')

        # Controllare se non è una pagina vuota
        no_result = soup.select('div[class*=ErrorLayout_container__]')
        if no_result:
            raise requests.RequestException('Non ci sono più pagine da analizzare.')

        products = soup.select('div.items__item.item-card')
        # pprint(products)

        items = []

        for adv in products:
            data = {
                'name': adv.select('h2[class*=ItemTitle-module_item-title__]'),
                'price': adv.select('p.price'),
                'town': adv.select('span[class*=index-module_town__]'),
                'city': adv.select('span.city'),
                'shipping': adv.select('span.shipping-badge'),
                'sold': adv.select('span.item-sold-badge'),
                # 'url': adv.select('a.link'),
            }

            for key, value in data.items():
                if value:
                    data[key] = value[0].get_text()
                else:
                    data[key] = None

                # data[key] = value[0].get_text() if value else None

            data.update({
                'price': float(data['price']
                            .replace('Spedizione disponibile','')
                            .replace('\xa0€', '')
                            .replace('Venduto', '')) if data['price'] else None,
                'url': adv.select('a.link')[0]['href'],
            })

            # [0].get_text().replace('Spedizione disponibile', '').replace('Venduto', ''),

            items.append(data)

    return items


curr_page = 1
all_data = []
max_page = input('Quante pagine vuoi analizzare? '
                 'Premi semplicemente invio per cercare su tutte le pagine disponibili.')

while True:
    if not max_page  or curr_page <= max_page:
    # if max_page is None or curr_page <= max_page:
        try:
            items = scrape_page(curr_page)
        except requests.RequestException as err:
            print(err)
            break
        
        all_data += items
        curr_page += 1
        sleep(1)
    else:
        break

pprint(all_data)

ImportError: attempted relative import with no known parent package